# 1. Word embedding là gì?

Ngôn ngữ là một hệ thống phức tạp được con người sử dụng để biểu đạt ngữ nghĩa. Trong đó, từ (word) là đơn vị cơ bản tạo nên một câu mang ngữ nghĩa qua đó ta có thể nói rằng một mô hình học máy hay mô hình học sâu sẽ không thể nào hiểu và học được để cho ra một kết quả dự đoán dựa trên danh sách các từ được. Vậy làm thế nào để mô hình tính toán có thể làm việc với ngôn ngữ tự nhiên? Câu trả lời của mình đó là biểu diễn các từ thành những con số để mô hình có thể học dựa trên chúng. Mình xin trình bày một số cách biểu diễn từ cơ bản như sau:
- **Biểu diễn mỗi từ bằng một con số**: đây có thể nói là một cách biểu diễn đơn giản nhất bằng cách ánh xạ mỗi từ vào một bộ từ điển với key là số và value là từ tương ứng. VD: "Tôi đi học" -> dictionary = {1: "đi", 2: "học", 3: "tôi"}.
- **Sử dụng one-hot vector**: Biểu diễn từ thành một vector với số lượng phần tử của vector bằng số lượng từ xuất hiện. Vector này có duy nhất một chiều và có giá trị bằng 1 ứng với từ đang biểu diễn, các vị trí khác có giá trị bằng 0. VD: [1, 0, 0, ..., 0].
- **Sử dụng vector ngẫu nhiên**: Mỗi từ được biểu diễn bàng một vector có giá trị của các chiều là ngẫu nhiên. VD: với 1000 từ chúng ta có thể biểu thị tất cả các từ đó trong không gian 3D, mỗi từ là một điểm trong không gian 3D đó.
- **Sử dụng word embedding**: Đây được coi là cách tốt nhất để thể hiện các từ trong văn bản tốt nhất và phổ biến nhất hiện nay. Kỹ thuật này cũng gán mỗi từ với một vector, nhưng điểm nổi bật hơn các kỹ thuật trên vì vector này được tính toán để biểu diễn quan hệ tương đồng giữa các từ.

Và trong notebook này chúng ta sẽ cùng tìm hiểu về word embedding và word2vec một phương pháp phổ biến trong word embedding. Word embedding là quá trình học biểu diễn cho một văn bản nơi mà các từ có sự tương đồng về nghĩa cũng sẽ có sự tương đồng về cách biểu diễn. Thực tế nó là một lớp các kỹ thuật nơi mà từng từ được biểu diễn là một vector với các phần tử trong vector là số thực (real-valued vector) trong một không gian vector đã được định nghĩa sẵn. Mỗi từ sẽ được ánh xạ đến một vector và giá trị của vector được học theo một cách tương tự như mạng neural network, do đó word embedding thường được áp dụng chung vào deep learning nói chung và trong NLP nói riêng.

Ý tưởng chính cảu word embedding là sử dụng một tầng ẩn (dense) biểu diễn sự phân phối cho mỗi từ. Mỗi từ được biểu diễn bởi một real-valued vector và thường lên đến hàng trục hay hàng trăm chiều. Điều này hoàn toàn trái ngược so với kỹ thuật one-hot encoding khi nó tạo ra mọt vector lên đến hàng triệu chiều để biểu diễn một từ. 



# 2. Mặt hạn chế của Vector One-Hot

Trước word embedding thì one-hot encoding được sử dụng cho việc mã hóa một từ. Nói một chút về one-hone encoding thì đây là phương pháp ánh xạ mỗi chỉ số thành một vector đơn vị khác nhau: Với chỉ số của một token sẽ nằm trong khoảng (0, N-1) với N là chiều dài của bộ từ vựng, trong đó vector của token thứ i tạo ra sẽ có độ dài là N các phần tử bằng 0 và phần tử ở vị trí thứ i sẽ bằng 1. Từ đó những vector này có thể được trực tiếp đưa vào mạng neural.

Như đã nói ở phần trên, vector one-hot sẽ có số chiều rất lớn, như vậy sẽ cần một tại nguyên lưu trữ lớn - đây cũng có thể coi là một hạn chế của vector one-hot. Điểm tiếp theo là mặc dù rất dễ xây dựng các vector one-hot, nhưng chúng thường không phải là lựa chọn tốt vì một trong những lý do đó là vector one-hot không thể biểu diễn chính xác độ tương đồng giữa các từ khác nhau (đây là điều mà word embedding là rất hiệu quả) chẳng hạn như độ tương đồng cosine.

![image](https://www.machinelearningplus.com/wp-content/uploads/2018/10/soft-cosine.png)

Do độ tương đồng cosine hai vector bất kỳ khác nhau đều bằng 0, nên rất khó sử dụng vector one-hot để biểu diễn độ tương đồng giữa các từ khác nhau.

# 3. Word2vec

Phương pháp word embedding học một real-valued vector đại diện cho một bộ từ điển đã được cố định về kích thước từ một văn bản. Quá trình học kết hợp với mô hình neural cho một số tác vụ như phân loại văn bản,... Chúng ta sẽ tìm hiểu một số kỹ thuật có thể được sử dụng để học một word embedding từ dữ liệu văn bản.

## Embedding layer
Một Embedding Layer là một tầng (layer) kết hợp với một mô hình mạng neural trên một tác vụ NLP cụ thể. Nó yêu cầu một văn bản cần phải được "clean" (loại bỏ stopword, ký hiệu đặc biệt, các dấu câu hay một số icon, ...). Kích thước của không gian vector (vector space) là một phần cụ thể của model thường là 50, 100, 300 chiều. Các vector được khởi tạo với một đại lượng ngẫu nhiên (thường rất nhỏ). Embedding layer được sử dụng phía trước của một mạng neural và những thông tin ở layer này đều sẽ được cập nhật với quá trình lan truyền ngược (Backpropagation). Nếu một mô hình Multilayer perceptron được sử dụng, những word vector này sẽ được nối lại trước khi "fed" với input cho model. Nếu sử dụng recurrent neural network, thì mỗi từ có thể được lấy làm một input trong một chuỗi (sequence). Cách tiếp cận này của embedding layer yêu cầu khá nhiều dữ liệu huấn luyện và việc huấn luyện sẽ chậm, nhưng nó sẽ học được một embedding cho tác vụ mà chúng ta cần xử lý.

## Word2Vec
Word2Vec là một phương pháp thống kê để đạt được sự hiệu quả cho quá trình học  word embedding từ một text corpus. Nó liên quan đến việc phân tích và khám phá toán học trên các vector biểu diễn của từ. Hai mô hình học tập khác nhau đã được giới thiệu có thể được sử dụng như một phần của Word2Vec đó là:
- Continuous Bag-of-Words (CBOW) model.
- Continuous Skip-Gram model.

CBOW model học một embedding từ việc dự đoán từ hiện tại dựa trên ngữ cảnh (context). Continuous skip-gram model học từ việc dự đoán từ những từ xung quanh từ hiện tại.

![image](https://www.researchgate.net/profile/Nailah-Al-Madi/publication/319954363/figure/fig1/AS:552189871353858@1508663732919/CBOW-and-Skip-gram-models-architecture-1.png)

Cả 2 models đều tập trung vào việc học dựa trên từ được nhận từ ngữ cảnh cục bộ được sử dụng (local usage context) nơi mà ngữ cảnh được đỉnh nghĩa bởi một chuỗi các từ lân cận. Và độ dài của chuỗi các từ lân cận này là một configurable parameter của model.

Lợi ích chính của cách tiếp cận này là đạt được hight-quality word embedding để việc học trở nên hiệu quả hơn (tối ưu về số chiều và thời gian tính toán), cho phép nhiều embedding hơn cho quá trình học từ một copus lớn (lên đến hàng tỷ từ).

## Mô hình Skip-gram
Skip-gram model giả định rằng một từ có thể được sử dụng để sinh ra các từ xung quanh nó trong một chuỗi văn bản. Ví dụ, giả sử chuỗi văn bản là ["học", "sinh", "trường", "học"]. Ta sử dụng từ "yêu" là từ trung tâm và đặt kích thước cho chuỗi ngữ cảnh xung quanh từ trung tâm đó là 2. 

![image](https://github.com/anminhhung/images/blob/main/skip-gram.png?raw=true)

Với từ trung tâm là "yêu" mô hình skip-gram quan tâm đến xác suất có điều kiện sinh ra các từ ngữ cảnh ["học", "sinh", "trường", "học"] nằm trong khoảng cách không quá 2 từ (đã thiết lập ở trên)
$$
P("học", "sinh", "trường", "học" | "yêu") (1)
$$

Giả định với từ trung tâm cho trước, các từ ngữ cảnh được sinh ra độc lập với nhau. Trong trường hợp này, công thức trên có thể viết lại thành 
$$
P("học" | "yêu").P("sinh" | "yêu").P("trường" | "yêu").P("học" | "yêu") (2)
$$

Trong skip-gram model, mỗi từ được biểu diễn bằng hai vector d-chiều để tính xác suất có điều kiện. Giả sử chỉ số của một từ trong từ điển là i, vector của từ được biểu diễn là $v_i \in R^d$ khi từ này là từ trung tâm và là $u_i \in R^d$ khi từ này là một từ ngữ cảnh. Gọi c và o lần lượt là chỉ số của từ trung tâm $w_c$ và từ ngữ cảnh $w_o$ trong từ điển. Có thể thu được xác suất có điều kiện sinh ra từ ngữ cảnh cho một từ trung tâm cho trước bằng phép toán softmax trên tích vô hướng của vector.
$$
P(w_o, w_c) = \frac{\exp(u_o^Tv_c)}{\sum_{i\in v}\exp(u_o^Tv_c)}
$$

Trong đó, tập chỉ số trong bộ từ vựng là $v = {1, 2, ..., |v|-1}$. Giả sử trong một chuỗi văn bản có độ dài T, tại thời điểm t được ký hiệu là $w^t$. Giả sử rằng các từ ngữ cảnh được sinh độc lập với từ trung tâm cho trước. Khi kích thước cửa sổ ngữ cảnh là m, hàm likelihood của skip-gram model là xác suất kết hợp sinh ra tất cả các từ ngữ cảnh với bất kỳ từ trung tâm cho trước nào.
$$
\prod_{t=1}^T\prod_{-m \leqslant j  \leqslant m, j \neq 0} P(w^{t+j}, w^t)
$$

Bạn xem thêm quá trình huấn luyện của skip-gram tại [**đây**](https://d2l.ai/chapter_natural-language-processing-pretraining/word2vec.html#training)

## Mô hình CBOW
Tương tự như mô hình skip-gram. Khác biệt lớn nhất là mô hình CBOW giả định từ trung tâm được tạo ra dựa trên các từ ngữ cảnh phía trước và sau nó trong một chuỗi văn bản. Với cùng một chuỗi văn bản ["học", "sinh", "trường", "học"], và từ "yêu" là từ trung tâm, đặt kích thước cho chuỗi ngữ cảnh xung quanh từ trung tâm đó là 2. CBOW quan tâm đến xác suất có điều kiện để sinh ra từ "yêu" dựa trên các từ ngữ cảnh "học", "sinh", "trường", "học".

![image](https://github.com/anminhhung/images/blob/main/CBOW.png?raw=true)

Với xác suất được xác định như sau:
$$
P("love"| "học", "sinh", "trường", "học") (3)
$$

Vì có quá nhiều từ ngữ cảnh trong mô hình CBOW, ta sẽ lấy trung bình các vector từ của chúng và sau đó sử dụng phương pháp tương tự như skip-gram để tính xác suất có điều kiện. Giả sử $v_i \in R^d$ và $u_i \in R^d$ là vector từ ngữ cảnh và vector từ trung tâm của từ có chỉ số i trong từ điển (ký hiểu ngược so với skip-gram). Gọi c là chỉ số của từ trung tâm $w_c$ và $o_1, ..., o_{2m}$ là chỉ số các từ ngữ cảnh $w_{o_1}, ..., W_{o_{2m}}$ trong từ điển. Do đó, xác suất có điều kiện sinh ra từ đích trung tâm dựa vào các từ ngữ cảnh cho trước là:

$$
P(w_c |w_{o_1}, ..., W_{o_{2m}}) = \frac{\exp(\frac{1}{2m}u_c^T(v_{o_1}+...+v_{o_{2m}}))}{\sum_{i \in v} \exp(\frac{1}{2m}u_i^T(v_{o_1}+...+v_{o_{2m}}))} (4)
$$

Để rút gọn (4), ký hiệu $W_o = {W_{o_1}, ..., w_{o_{2m}}}$ và $	\bar{v}=(v_{o_1}+...+v_{o_{2m}})/(2m)$. (4) trở thành:
$$
P(w_c|W_o) = \frac{exp(U_c^T\bar{v}_o)}{\sum_{i \in v}\exp(u_i^T\bar{v}_o)}
$$

Cho một chuỗi văn bản có độ dài T, ta giả định từ xuất hiện tại bước thời gian t và $w^t$ và kích thước của cửa sổ ngữ cảnh là m. Hàm mục tiêu của mô hình CBOW là xác suất sinh ra bất kỳ từ đích trung tâm nào dựa vào những từ ngữ cảnh.

$$
\prod_{t=1}^TP(w^t|w^{t-m}, ..., w^{t-1}, w^{t+1}, ..., w^{t+m})
$$

Phần huấn luyện mô hình CBOW xem tại [**đây**](https://d2l.ai/chapter_natural-language-processing-pretraining/word2vec.html#id3)

# Code demo

In [8]:
import torch
from torch import nn, optim
import torch.nn.functional as F

## CBOW

In [28]:
class CBOW(nn.Module):
  def __init__(self, vocabulary_size, embedding_features, padding_idx=0):
    super(CBOW, self).__init__()
    self.embedding = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_features, padding_idx = padding_idx)
    self.fc = nn.Linear(in_features = embedding_features, out_features = vocabulary_size)
  
  def forward(self, x):
    x = self.embedding(x)
    x = x.sum(dim=1) / x.shape[1]
    log_prob = F.log_softmax(self.fc(x), dim=1).unsqueeze(1)

    return log_prob

## Skip-gram

In [3]:
class SKIP_GRAM(nn.Module):
    def __init__(self, vocabulary_Size, embedding_features, context_len, padding_idx=0 ):
        super(SKIP_GRAM, self).__init__()
        self.context_len = context_len
        self.embedding = nn.Embedding(num_embeddings = vocabulary_Size, embedding_dim=embedding_features, padding_idx=padding_idx)
        self.fc = nn.Linear(in_features = embedding_features, out_features = vocabulary_Size)
    
    def forward(self, x):
        context_out = []
        for i in range(self.context_len):
            x_ = self.embedding(x)
            context_word_i = self.fc(x_)
            context_out.append(context_word_i)
        log_prob = F.log_softmax(torch.stack(context_out, dim=1).squeeze(), dim=1)
        return log_prob

## So sánh sự khác nhau giữa squeeze và unsqueeze

![image](https://i.stack.imgur.com/9AJJA.png)